### 데이터 읽어오기

In [11]:
import glob
import os
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [12]:
txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

In [13]:
raw_corpus = []

#여러개의 txt파일을 모두 읽어서 raw_corpus에 담는다.
for txt_file in txt_list:
    with open(txt_file, 'r') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기: ", len(raw_corpus))
print("Examples : \n", raw_corpus[:3])

데이터 크기:  187088
Examples : 
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


### 데이터 정제

In [14]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r"\1", sentence)
    sentence = re.sub(r'[" "]+'," ",sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+"," ",sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

#test
print(preprocess_sentence("This @_is ;;;;sample          sentence."))

<start> this is sample sentence. <end>


In [15]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0 : continue
    if sentence[-1] == ':' : continue
    temp = preprocess_sentence(sentence)
    if len(temp.split()) > 15 : continue
    corpus.append(temp)
        
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
#10개만 테스트
corpus[:10]

['<start> hook <end>',
 '<start> hook <end>',
 '<start> i ve been down so long, it look like up to me <end>',
 '<start> i ve been down so long, it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face, straight up to my face <end>',
 '<start> straight up to my face, straight up to my face <end>']

In [16]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 12000, filters = '', oov_token = "<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

src_input = tensor[:,:-1]
tgt_input = tensor[:, 1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)

In [17]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (256488, 14)
Target Train: (256488, 14)


### 인공지능 만들기

In [18]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)  # 입력된 텐서에는 단어사전의 인덱스가 들어있는데, 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out
    
embedding_size = 19
hidden_size = 2048
model = TextGenerator(tokenizer.num_words+1, embedding_size, hidden_size)

In [19]:
history = []
epochs = 10

optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

history = model.fit(enc_train, 
          dec_train, 
          epochs=epochs,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

Epoch 1/10
1002/1002 [==============================] - 891s 864ms/step - loss: 3.6128 - val_loss: 2.8604
Epoch 2/10
1002/1002 [==============================] - 865s 863ms/step - loss: 2.7340 - val_loss: 2.5133
Epoch 3/10
1002/1002 [==============================] - 865s 863ms/step - loss: 2.3068 - val_loss: 2.2169
Epoch 4/10
1002/1002 [==============================] - 868s 867ms/step - loss: 1.9385 - val_loss: 1.9838
Epoch 5/10
1002/1002 [==============================] - 861s 859ms/step - loss: 1.6611 - val_loss: 1.8060
Epoch 6/10
1002/1002 [==============================] - 864s 863ms/step - loss: 1.4649 - val_loss: 1.6687
Epoch 7/10
1002/1002 [==============================] - 866s 864ms/step - loss: 1.3309 - val_loss: 1.5737
Epoch 8/10
1002/1002 [==============================] - 861s 859ms/step - loss: 1.2355 - val_loss: 1.5007
Epoch 9/10
1002/1002 [==============================] - 865s 863ms/step - loss: 1.1667 - val_loss: 1.4523
Epoch 10/10
1002/1002 [=======================

In [23]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [24]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '